In [4]:
import pandas as pd
import os,glob
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import sys
sys.path.append(os.path.expanduser("~") + "/collproc/")
from raw import *
from coll import *
import re

In [5]:
df = unpkl('raw.pkl')

# Overall standard deviation

In [6]:
# allmods = [item for sublist in df.thickness for item in sublist]
# alltrads = [item for sublist in lf.thickness for item in sublist]
# allthicknesses = allmods + alltrads

In [3]:
# thickstd = np.std(allthicknesses)
# thickstd

# Corrections

# Modern dupes

### Enumeration of Cases

In [ ]:
from itertools import combinations

In [ ]:
moddupes = df.catalog.value_counts().loc[lambda x: x > 1]
for moddupe in moddupes.index:
    tmp = df[df.catalog==moddupe]
    
    # we exclude case where both Leah and I measured binder 4
    tmp = tmp.sort_values('fname')
    fname_string = '|'.join(list(tmp.fname))
    
    # we exclude case where `catalog_manbran` is the same and the medians are within 1SD
    l = list(tmp.thickness.apply(np.median))
    medpairs = list(combinations(l, 2))
    meddiffs = [abs(item[0]-item[1]) for item in medpairs]
    
    if 'thickness_dc_binder4.xlsx|thickness_ll_binder4' in fname_string: # hers are spread across multiple files
        continue
    elif all([diff < thickstd for diff in meddiffs]) and len(tmp.catalog_manbran.unique())==1:
        continue
    else:
        print(moddupe,end='\t')

In [ ]:
df[df.catalog=='5155']

In [ ]:
lf[lf.catalog=='5155']